In [11]:
# coding: utf-8
import scipy.io as io
from scipy import optimize
import numpy as np
import math
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
plt.gray();
import matplotlib.patheffects as pe
import torch
import torchvision
import torchvision.transforms as transforms
import seaborn as sb
from sklearn.manifold import TSNE 
from numpy import *
from sklearn.decomposition import PCA
%matplotlib inline
# from IPython.display import display
# np.set_printoptions(suppress=True, threshold=np.nan)    #去除科学计数法
plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus'] = False # 显示中文

transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])
train_data=torchvision.datasets.CIFAR10(root='./',train=True,
                                        download=False,transform=transform)
test_data=torchvision.datasets.CIFAR10(root='./',train=False,
                                        download=False,transform=transform)


In [18]:
def getUniformData(X,Y,n):
    n_per_class=int(n/10)
    X_uniform=[]
    Y_uniform=[]
    for i in range(10):
        Xi=X[Y[:,0] == i]
        randIndices=[np.random.randint(0, Xi.shape[0]) for i in range(n_per_class)]
        X_uniform.append(Xi[randIndices])
        Yi=Y[(Y[:,0]==i)]
        Y_uniform.append(Yi[randIndices])
    X_uniform=np.vstack(np.array(X_uniform))
    Y_uniform=np.vstack(np.array(Y_uniform))
    return X_uniform, Y_uniform

def rgb2gray(images):
    return np.expand_dims(np.dot(images, [0.2990, 0.5870, 0.1140]), axis=3)

In [23]:
THREASHOLD=0.5
n=1000
n_test=200
dopca=1

X=train_data.data
y=np.array(train_data.targets)[:, np.newaxis]
X_test=test_data.data
y_test=np.array(test_data.targets)[:, np.newaxis]
print(X.shape,y.shape,X_test.shape,y_test.shape)
print(np.unique(y))

X = rgb2gray(X).astype(np.float32)
X, y = getUniformData(X,y,n)
X=X.reshape(n,-1)  
X_mean = np.mean(X, axis=0)  # 第一维变化，对其他维每个位置上求均值和方差
X_std = np.std(X, axis=0)
X = (X - X_mean) / X_std
if dopca:
    pca = PCA(n_components=100)
    pca.fit(X)                  #训练
    X=pca.fit_transform(X)   #降维后的数据
d=X.shape[1]

X_test = rgb2gray(X_test).astype(np.float32)
X_test, y_test = getUniformData(X_test, y_test,n_test)
X_test=X_test.reshape(n_test,-1)
X_test = (X_test - X_mean) / X_std
if dopca:
    pca_test = PCA(n_components=d)
    pca_test.fit(X_test)                  #训练
    X_test=pca_test.fit_transform(X_test)   #降维后的数据

print(X.shape,y.shape,X_test.shape,y_test.shape)
print(np.sum(pca.explained_variance_ratio_),np.sum(pca_test.explained_variance_ratio_))

class LogsiticRegression(object):
    w = None # 权重
    W = None
    n = 0 # 样本个数
    d = 0 # 维度个数
    learning_rate = 0.01
    loss_difference = 0.001 # loss 改变的百分比
    loss_bound = 0.3
    num_labels = 10
    plambda = 0.2 # 正则参数
    
    def __init__(self,*args, **kwargs):
        self.f=open('output.txt', "w",encoding='utf-8')
        
    def sigmoid(self,xb): # xb 是列向量
#         s=np.ones((len(xb),1))   
#         s=1.0/(1.0+np.exp(-xb))
#         return s
        return 1.0 / (1.0 + np.exp(-xb))

    def loss(self,X,Y):
        Xw=self.sigmoid(np.dot(X,self.w))
#         w2=self.w.copy()  
#         w2[0]=0           
        Loss=-(np.dot(Y.transpose(),np.log(Xw))+np.dot(np.transpose(1-Y),np.log(1-Xw)))/self.n # Xw里有1导致log 0
        #+(plambda*np.dot(np.transpose(w2),w2))/(2*self.n) 
        return Loss[0][0]
    
    # 课件1 P4 简化公式
    def loss2(self,X,Y):
        Xw=np.dot(X,self.w) 
#         w2=w.copy()  
#         w2[0]=0           
        Loss=(np.sum(np.log(1+np.exp(Xw)))-np.dot(Y.transpose(),Xw))/self.n
        #+(plambda*np.dot(np.transpose(w2),w2))/(2*self.n) 
        return Loss[0][0]
    
    # 逐个样本计算,去除异常值
    def loss3(self,X,Y):
        sum_err = 0.0
        Xw=self.sigmoid(np.dot(X,self.w))  #  Bug： 一直没加sigmoid！！！
        for i in range(self.n):
            if Xw[i][0] > 0 and Xw[i][0] < 1:
                sum_err -= (Y[i][0] * np.log(Xw[i][0]) + (1-Y[i][0]) * np.log(1-Xw[i][0]))
        return sum_err / self.n
    
    def loss4(self,w,X,Y,initial_lambda):
        n=len(Y)
        h=1.0/(1.0+np.exp(-np.dot(X,w)))
#         theta1=w.copy()    
#         theta1[0]=0         
        J=-(np.dot(np.transpose(Y),np.log(h))+np.dot(np.transpose(1-Y),np.log(1-h)))/self.n #+(initial_lambda*np.dot(np.transpose(theta1),theta1))/(2*n)
        return J
    
    def loss5_ridge(self,X,Y):
        y_Xw=Y-np.dot(X,self.w)
        return (np.dot(y_Xw.transpose(),y_Xw)[0][0]+self.plambda*np.dot(self.w.transpose(),self.w)[0][0])/self.n

    def loss6_lasso(self,X,Y):
        y_Xw=Y-np.dot(X,self.w)
        return (np.dot(y_Xw.transpose(),y_Xw)[0][0]/2+self.plambda*np.sum(abs(self.w)))/self.n
    
    def gradient(self,X,Y):
#         g=np.zeros((self.d))
        hx=self.sigmoid(np.dot(X,self.w))    #调用sigmoid函数
        w2=self.w.copy()
        w2[0]=0            
        g=np.dot(np.transpose(X),hx-Y)+self.plambda*w2#/self.n # 加第二项就是ridge logistic regression
        return g
    
    def gradient2(self,w,X,Y,initial_lambda):
#         g=np.zeros((self.d))
        n=X.shape[0]
        hx=self.sigmoid(np.dot(X,w))
#         w2=w.copy()
#         w2[0]=0            
        g=np.dot(np.transpose(X),hx-Y)/n
        #+initial_lambda*w2/n
        return g
    
    def gradient5_ridge(self,X,Y):
        # −2X(Y −Xˆβλ) + 2λˆβλ.
        a=np.dot(X.transpose(),Y-np.dot(X,self.w))
#         print(a[0:20])
        return -2*a/np.max(a)+2*self.plambda*self.w
         
    def train(self,X,Y):
        self.n = X.shape[0]
        self.d = X.shape[1]
        y_class=np.zeros((X.shape[0],1))  # 第i个样本是不是数字j
        self.W = np.zeros((X.shape[1],self.num_labels))
        
        for i in range(self.num_labels):
            count = 1
            new_loss=0
            self.learning_rate=0.01
            self.w = np.full((self.d,1),1/self.d) # np.zeros((X.shape[1],1))
            y_class[:,0]=np.int32(Y==i).reshape(1,-1) # 行向量赋给列向量
            acc=0
            w_max = np.full((self.d,1),1/self.d)
            acc_max=0
            while True:
                old_loss = new_loss
                self.w = self.w - self.learning_rate * self.gradient(X,y_class)#/np.max(self.w) # gradient5_ridge
                acc,aoc=self.prediect_one_class(X,y_class)
                
#                 self.w=self.w/np.max(self.w)
                new_loss = self.loss3(X, y_class) # loss3 loss5_ridge
                if acc>0.8 or count>500 or (new_loss < self.loss_bound and math.fabs(old_loss - new_loss)/max(self.loss_bound,new_loss) < self.loss_difference):
                    print("第"+str(i)+"个分类器训练完毕, ","收敛到:",new_loss)
#                     self.f.write("第"+str(i)+"个分类器训练完毕\n收敛到:"+str(new_loss)+"\n")
                    break
                if acc>acc_max:
                    acc_max=acc
                    w_max=self.w
                    print("训练数字"+str(i)+" 迭代第",count,"次, loss=",new_loss,"acc_max=",acc_max,"大w个数 ",np.sum(self.w>1000000000))
#                     print(self.w[0:10,0])
                    
#                     print(self.w)
#                 self.f.write("训练数字"+str(i)+" 迭代第"+str(count)+"次!\n"+str(old_loss)+'\n'+str(new_loss)+'\n')
#                 print("损失减小:",(old_loss - new_loss))
#                 print(old_loss)
                count += 1
                self.learning_rate=self.learning_rate*0.99
                
            print("数字"+str(i)+"训练完毕")
#             self.f.write("数字"+str(i)+"训练完毕\n")
            self.W[:,i]=w_max[:,0]
#             self.W[:,i]=self.W[:,i-1]/np.max(self.W[:,i-1])
 
    def train_Lasso(self,X,Y):
        self.n = X.shape[0]
        self.d = X.shape[1]
        y_class=np.zeros((X.shape[0],1))  # 第i个样本是不是数字j
        self.W = np.zeros((X.shape[1],self.num_labels))
        
        
        for i in range(self.num_labels):
            count = 1
            new_loss=0
            delta=1
            self.w = np.full((self.d,1),1.0) # np.zeros((X.shape[1],1))
            y_class[:,0]=np.int32(Y==i).reshape(1,-1) # 行向量赋给列向量
            acc=0
            R = np.full((self.n,self.d),0.1)
            r = np.full((self.d,1),0.01)
            w_max = np.full((self.d,1),1.0)
            acc_max=0
            for q in range(1,21):
                for j in range(self.d):
                    w=self.w.copy()
                    w[j][0]=0
                    R[:,j]=(y_class-np.dot(X,w))[:,0]
                    xx=np.sum(np.dot(X[:,j],X[:,j]))
                    r[j][0]=np.sum(np.dot(R[:,j],X[:,j]))/xx
#                     print(r[j][0])
#                     print(delta/xx)
                    self.w[j][0]=r[j][0]/abs(r[j][0])*max(0,abs(r[j][0])-delta/xx)
                    acc,aoc=self.prediect_one_class(X,y_class)
                    if acc_max<acc:
                        acc_max=acc
                        w_max=self.w
                        print("数字",i,", 第",q,"轮 loss: ",self.loss6_lasso(X,Y),"|w|.sum: ",np.sum(abs(self.w)),"acc= ",acc)
                #self.w=self.w/np.max(self.w)
                delta=delta*0.9

                    
            self.W[:,i]=w_max[:,0]
            print("数字"+str(i)+"训练完毕")

    # 多分类 同时预测每个类的准确率，选最大值
    def predict(self,X_test,y_test):
        y_predict = self.sigmoid(np.dot(X_test,self.W))
        y_class = np.zeros((X_test.shape[0],1))
        for i in range(X_test.shape[0]):
#             max_class=0
#             for j in range(self.num_labels):
#                 if y_predict[i][j] >0.5:
            max_class=0
            for j in range(1,self.num_labels):
                if y_predict[i][j] >y_predict[i][max_class]:
                    max_class=j
                    break
            y_class[i][0]=max_class
        result = (y_test==y_class)
        print("最大式多分类总和准确率："+str(np.sum(result)/len(result)))
#         print("auc: ", roc_auc_score(y_test, y_predict,average='macro'))#multi_class='ovo',average='weighted'
        
#         self.f.write("准确率："+str(np.sum(result)/len(result))+'\n')
        return np.sum(result)/len(result)

    # 二分类 输入为向量
    def prediect_one_class(self,X,Y): 
        y_predict = self.sigmoid(np.dot(X,self.w))
        y_class = np.zeros((X.shape[0],1))
        for i in range(X.shape[0]):
            if y_predict[i][0]>THREASHOLD:
                y_class[i][0]=1
        result = (Y==y_class)
        acc=np.sum(result)/len(result)
        return acc,roc_auc_score(Y,y_class)
    
    # 二分类 逐个预测每个类的准确率
    def predict2(self,X,Y):
        y_class = np.zeros((X.shape[0],1)) # 预测出是不是类i
        Y_class = np.zeros((X.shape[0],1)) # 真实类是不是类i
        acc=[]
        auc=[]
        for i in range(self.num_labels):      
            y_predict = self.sigmoid(np.dot(X,self.W[:,i]))[:, np.newaxis] # 扩充一维
            y_class = np.zeros((X.shape[0],1))
            for j in range(X.shape[0]):
                if y_predict[j][0]>THREASHOLD:
                    y_class[j][0]=1
            Y_class[:,0]=np.int32(Y==i).reshape(1,-1) # 行向量赋给列向量
            result = (Y_class==y_class)
            acc.append(np.sum(result)/len(result))
            auc.append(roc_auc_score(Y_class,y_class))
        
        wish_acc=0
        t=1
        for i in range(10):
            wish_acc+=t*acc[i]
            t*=acc[i]
        
        print("每个类上的准确率：",acc,"迭代式应得准确率：",wish_acc/10,"每个类上的auc: ",auc)     

    # 多分类 同时预测每个类的准确率, 迭代判断
    def predict3(self,X,Y):
        y_predict = self.sigmoid(np.dot(X,self.W))
        y_class = np.zeros((X.shape[0],1))
        for i in range(X.shape[0]):
            max_class=0
            for j in range(self.num_labels):
                if y_predict[i][j] >THREASHOLD:
#             max_class=1
#             for j in range(1,self.num_labels):
#                 if y_predict[i][j] >y_predict[i][max_class]:
                    max_class=j
                    break
            y_class[i][0]=max_class
        result = (Y==y_class)
        print("迭代式多分类总和准确率："+str(np.sum(result)/len(result)))
#         print("auc: ",roc_auc_score(Y,y_predict,multi_class='ovo'))
#         self.f.write("准确率："+str(np.sum(result)/len(result))+'\n')
        return np.sum(result)/len(result)
    
    def save_W(self):
        with open('W.npy', 'wb') as f:
            np.save(f, self.W)
            f.close()
    def load_W(self):
        with open('W.npy', 'rb') as f:
            self.W = np.load(f)
            f.close()
        np.savez('./model.npz', L0_W = session.run(L0_W), L0_B = session.run(L0_B))
        
lr = LogsiticRegression()
# lr.train(X,y)
lr.train_Lasso(X,y)
print("训练集")
lr.predict(X,y)
lr.predict2(X,y)
lr.predict3(X,y)
print("\n测试集")
lr.predict(X_test,y_test)
lr.predict2(X_test,y_test)
lr.predict3(X_test,y_test)
# lr.save_W()
# lr.f.close()

(50000, 32, 32, 3) (50000, 1) (10000, 32, 32, 3) (10000, 1)
[0 1 2 3 4 5 6 7 8 9]
(50000, 32, 32, 1) (50000, 1)
(10000, 32, 32, 1) (10000, 1)
(1000, 100) (1000, 1) (200, 100) (200, 1)
0.9303927 0.97232157
数字 0 , 第 1 轮 loss:  346.0402949840355 |w|.sum:  99.00363766127649 acc=  0.486
数字 0 , 第 1 轮 loss:  280.1333567507494 |w|.sum:  98.00423823834325 acc=  0.492
数字 0 , 第 1 轮 loss:  239.51450554219008 |w|.sum:  97.0050598163731 acc=  0.507
数字 0 , 第 1 轮 loss:  130.5136115149717 |w|.sum:  88.02529486665824 acc=  0.509
数字 0 , 第 1 轮 loss:  124.40915795157858 |w|.sum:  87.02900929910274 acc=  0.512
数字 0 , 第 1 轮 loss:  115.1962582441023 |w|.sum:  85.03175869338946 acc=  0.513
数字 0 , 第 1 轮 loss:  102.6175905386151 |w|.sum:  82.03428485009934 acc=  0.52
数字 0 , 第 1 轮 loss:  59.32104486108912 |w|.sum:  62.10353093710428 acc=  0.529
数字 0 , 第 1 轮 loss:  50.19580242656709 |w|.sum:  55.1317760888398 acc=  0.53
数字 0 , 第 1 轮 loss:  43.98898405198839 |w|.sum:  49.16134139952153 acc=  0.532
数字 0 , 第 1 轮 loss

数字 8 , 第 1 轮 loss:  17.856960428906373 |w|.sum:  10.339226114109891 acc=  0.541
数字 8 , 第 1 轮 loss:  14.211664134652665 |w|.sum:  0.4484935345607652 acc=  0.61
数字 8 , 第 2 轮 loss:  14.211606200242972 |w|.sum:  0.45147334940252376 acc=  0.611
数字 8 , 第 3 轮 loss:  14.2114805388373 |w|.sum:  0.4556698146043569 acc=  0.612
数字 8 , 第 3 轮 loss:  14.211463723897662 |w|.sum:  0.45629117328683144 acc=  0.613
数字8训练完毕
数字 9 , 第 1 轮 loss:  346.0351393587786 |w|.sum:  99.00074315974649 acc=  0.478
数字 9 , 第 1 轮 loss:  280.11410851511465 |w|.sum:  98.00652332301726 acc=  0.494
数字 9 , 第 1 轮 loss:  239.49582239697892 |w|.sum:  97.01018294683976 acc=  0.529
数字 9 , 第 1 轮 loss:  185.35837880744228 |w|.sum:  94.01771184142903 acc=  0.531
数字 9 , 第 1 轮 loss:  93.07571202206539 |w|.sum:  79.05946317329 acc=  0.54
数字 9 , 第 1 轮 loss:  14.170895048025132 |w|.sum:  0.5472458783050526 acc=  0.622
数字 9 , 第 2 轮 loss:  14.170892515252106 |w|.sum:  0.5472508024949142 acc=  0.623
数字9训练完毕
训练集
最大式多分类总和准确率：0.243
每个类上的准确率： [0.6

0.105